In [1]:
import os

os.chdir('../ImageBind')

from imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType

from torch.utils.data import DataLoader


import pandas as pd
import torch

import torch.nn as nn
import torch.optim as optim


/home/ethanyu/miniconda3/envs/imagebind/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ethanyu/miniconda3/envs/imagebind/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/ethanyu/miniconda3/envs/imagebind/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=False)
model.train()
model.to(device)

In [ ]:
os.chdir("../Temporal-ImageBind")
df = pd.read_csv("data/Temporal-Imagebind/imu/")
num_rows = len(df)
tensors = []
for start_row in range(0, num_rows, 100):
    subset = df.iloc[start_row:start_row+100, -7:]
    tensor = torch.tensor(subset.values)
    transposed_tensor = tensor.transpose(0, 1)
    tensors.append(transposed_tensor)


inputs = {
    ModalityType.VISION: data.load_and_transform_vision_data("data/Temporal-Imagebind/images/", device),
    ModalityType.IMU: tensors
}

In [ ]:
train_loader = ...

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.2):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, q, k):
        similarity = torch.matmul(q, k.t()) / self.temperature
        
        numerator = torch.exp(torch.diag(similarity))
        denominator = torch.sum(torch.exp(similarity), dim=1) + numerator
        
        # Calculate the loss
        loss = -torch.mean(torch.log(numerator / denominator))
        
        return loss


In [ ]:
loss_function = ContrastiveLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01) 
num_epochs = 100

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs1, inputs2, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients

        outputs1 = model(inputs1)
        outputs2 = model(inputs2)
        
        loss = loss_function(outputs1, outputs2, labels)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print average loss per epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")